In [1]:
import subprocess, time
from pathlib import Path

In [2]:
# beast-0.10.2-SNAPSHOT/bin/beast --jars beast-examples-0.10.0-SNAPSHOT.jar rdprofilter Landsat8_Riverside filterpixel.tif

In [3]:
from pathlib import Path

BEAST_HOME = Path("/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT")
BEAST_LIB = BEAST_HOME / "lib"

DEPS = [str(jar) for jar in BEAST_LIB.glob("*.jar")]

print("Beast jars:")
for j in DEPS:
    print(" ", j)

Beast jars:
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/satex-0.10.2-SNAPSHOT.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/jai_core-1.1.3.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/visualization-0.10.2-SNAPSHOT.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/dataExplorer-0.10.2-SNAPSHOT.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/io-0.10.2-SNAPSHOT.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/common-0.10.2-SNAPSHOT.jar
  /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10

In [4]:
# POSIX classpath (macOS / Linux)
COMPILE_CLASSPATH = ":".join(DEPS)

# Spark --jars expects comma-separated
RUNTIME_JARS = ",".join(DEPS)

print("Compile classpath:\n", COMPILE_CLASSPATH)
print("\nRuntime jars:\n", RUNTIME_JARS)

Compile classpath:
 /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/satex-0.10.2-SNAPSHOT.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/jai_core-1.1.3.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/visualization-0.10.2-SNAPSHOT.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/dataExplorer-0.10.2-SNAPSHOT.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/io-0.10.2-SNAPSHOT.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/common-0.10.2-SNAPSHOT.jar:/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNA

In [37]:
from pathlib import Path
import subprocess, shlex

RUN_DIR = Path("./runs/ndvi_eval")
RUN_DIR.mkdir(parents=True, exist_ok=True)

SRC_SCALA = Path("/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/notebook/runs/workspace/Job.manual.codeExample.scala")
COMBINED = RUN_DIR / "ndvi_test.scala"
LOG_FILE = RUN_DIR / "ndvi_test_spark_shell.log"

IVY_DIR = (RUN_DIR / ".ivy2").resolve()
IVY_DIR.mkdir(parents=True, exist_ok=True)

# --- build combined scala script ---
src_text = SRC_SCALA.read_text(encoding="utf-8")

# remove spark-shell directives if any exist in the source
cleaned = "\n".join(
    line for line in src_text.splitlines()
    if not line.lstrip().startswith(":")
).rstrip()

# IMPORTANT: actually call the step, then exit
combined_text = cleaned + "\n\n" + "\n".join([
    "val _r = ndvi.run(sc)",
    'println("__DONE__")',
    "System.exit(0)",
]) + "\n"

COMBINED.write_text(combined_text, encoding="utf-8")
print("Wrote combined:", COMBINED.resolve())

# --- Beast jars ---
BEAST_HOME = Path("../beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT").resolve()
beast_lib = BEAST_HOME / "lib"
beast_jars = sorted(str(p) for p in beast_lib.glob("*.jar"))
RUNTIME_JARS = ",".join(beast_jars)

# --- spark-shell invocation ---
SPARK_SHELL = "/Users/clockorangezoe/miniconda3/envs/geo_llm_spark/bin/spark-shell"

repos = ",".join([
    "https://repo.osgeo.org/repository/geotools-releases/",
    "https://repo.osgeo.org/repository/release/",
])

packages = ",".join([
    "org.locationtech.jts:jts-core:1.19.0",
    "org.geotools:gt-referencing:24.1",
    "org.geotools:gt-epsg-hsql:24.1",
    "org.geotools:gt-geotiff:24.1",
    "org.geotools:gt-coverage:24.1",
    "it.geosolutions.imageio-ext:imageio-ext-tiff:1.4.14",
])

Wrote combined: /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/notebook/runs/ndvi_eval/ndvi_test.scala


In [38]:
cmd = [
    SPARK_SHELL,
    "--master", "local[*]",
    "--conf", "spark.hadoop.fs.defaultFS=file:///",
    "--conf", "spark.hadoop.fs.file.impl=org.apache.hadoop.fs.LocalFileSystem",
    "--conf", f"spark.jars.ivy={IVY_DIR}",
    "--repositories", repos,
    "--packages", packages,
    "--jars", RUNTIME_JARS,
    "-i", str(COMBINED),
]

print("Running:\n", " ".join(map(shlex.quote, cmd)))
p = subprocess.run(cmd, text=True, capture_output=True)

LOG_FILE.write_text(p.stdout + "\n\n--- STDERR ---\n" + p.stderr, encoding="utf-8")
print("Log:", LOG_FILE.resolve())

# 4) Robust success check
out_lower = (p.stdout + "\n" + p.stderr).lower()
bad_markers = [
    "exception in thread",
    "sparkexception",
    "noclassdeffounderror",
    "classnotfoundexception",
    "unresolved dependency",
    "error executor",
    "error tasksetmanager",
    "error:",
]
ok = (p.returncode == 0) and not any(m in out_lower for m in bad_markers)

print("Return code:", p.returncode, "OK:", ok)
print("\n--- Last 60 lines ---\n")
print("\n".join((p.stdout + "\n" + p.stderr).splitlines()[-60:]))

ok

Running:
 /Users/clockorangezoe/miniconda3/envs/geo_llm_spark/bin/spark-shell --master 'local[*]' --conf spark.hadoop.fs.defaultFS=file:/// --conf spark.hadoop.fs.file.impl=org.apache.hadoop.fs.LocalFileSystem --conf spark.jars.ivy=/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/notebook/runs/ndvi_eval/.ivy2 --repositories https://repo.osgeo.org/repository/geotools-releases/,https://repo.osgeo.org/repository/release/ --packages org.locationtech.jts:jts-core:1.19.0,org.geotools:gt-referencing:24.1,org.geotools:gt-epsg-hsql:24.1,org.geotools:gt-geotiff:24.1,org.geotools:gt-coverage:24.1,it.geosolutions.imageio-ext:imageio-ext-tiff:1.4.14 --jars /Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/beast-spark-0.10.2-SNAPSHOT.jar,/Users/clockorangezoe/Documents/phd_projects/code/geoAI/RDProLLMagent/beast-0.10.2-SNAPSHOT-bin/beast-0.10.2-SNAPSHOT/lib/cg-0.10.2-SNAPSHOT.jar,/Users/clockorangezoe/Docume

True